In [ ]:
import sys
sys.path.append('..')
from config_figures import *

In [ ]:
df = pd.read_csv(validation_stats_file).drop(['.geo'], axis=1).rename(columns = {'system:index': 'species'}).sort_values('nobs', ascending=False)
df['nobs_cat'] = ['at least 90' if x >= 90 else 'less than 90' for x in df['nobs']] 
df

In [ ]:
df_melt = pd.melt(df, id_vars=['species', 'nobs', 'nobs_cat'] , value_vars=['tss','precision','recall'], var_name='metric')
df_melt

In [ ]:
g = sns.relplot(df_melt, x='nobs', y='value', hue='nobs_cat', col='metric', s=3, alpha=0.5, edgecolor="none", height=4, legend=False)
g.set_titles(col_template="{col_name}")
g.set(xlabel='Number of occurrences')
g.axes[0,0].set_title('True skill statistic (TSS)')
g.axes[0,1].set_title('Precision')
g.axes[0,2].set_title('Recall')
plt.savefig(figuredir + 'nobs_tss_precision_recall.png') 

In [ ]:
g = sns.FacetGrid(df_melt[df_melt['nobs'] >= 90], col="metric", sharex=False)
g.map(sns.histplot, 'value', bins = 25, edgecolor=None)
g.set_titles(col_template="")
g.axes[0,0].set_xlabel('True skill statistic')
g.axes[0,1].set_xlabel('Precision')
g.axes[0,2].set_xlabel('Recall')
plt.savefig(figuredir + 'tss_precision_recall.png')

In [ ]:
df_splot = pd.read_csv(sdm_splot_file).drop(['system:index', '.geo'], axis=1)
df_splot = df_splot.merge(df[['species','nobs']], on='species')
df_splot['tss'] = (df_splot['tp'] / (df_splot['tp'] + df_splot['fn'])) + (df_splot['tn'] / (df_splot['tn'] + df_splot['fp'])) - 1
df_splot['precision'] = df_splot['tp'] / (df_splot['tp'] + df_splot['fp'])
df_splot['recall'] = df_splot['tp'] / (df_splot['tp'] + df_splot['fn'])
df_splot['f1'] = (2 * df_splot['precision'] * df_splot['recall']) / (df_splot['precision'] + df_splot['recall'])
df_splot

In [ ]:
df_splot_5pos = df_splot[df_splot[['fn','tp']].sum(axis=1) > 5]
df_splot_5pos

In [ ]:
print(df_splot_5pos['tss'].mean(), df_splot_5pos['tss'].std())

In [ ]:
# average false positive rate
(df_splot_5pos['fp'] / (df_splot_5pos['fp'] + df_splot_5pos['tn'])).mean() 

In [ ]:
# average false negative rate
(df_splot_5pos['fn'] / (df_splot_5pos['fn'] + df_splot_5pos['tp'])).mean() 

In [ ]:
df_splot_melt = pd.melt(df_splot_5pos, id_vars=['species','nobs'], value_vars=['tss','precision','recall'], var_name='metric')
df_splot_melt

In [ ]:
g = sns.FacetGrid(df_splot_melt, col="metric", sharex=False, sharey=False)
g.map(sns.histplot, 'value', bins = 25, edgecolor=None)
g.set_titles(col_template="")
g.axes[0,0].set_xlabel('True skill statistic')
g.axes[0,1].set_xlabel('Precision')
g.axes[0,2].set_xlabel('Recall')
plt.savefig(figuredir + 'splot_tss_precision_recall.png')

In [ ]:
sdm_mhs_df = pd.read_csv(sdm_mhs_iou_file).drop(columns=['.geo'])
sdm_mhs_df = sdm_mhs_df[sdm_mhs_df['IoU'] != 0].sort_values('IoU')
sdm_mhs_df['system:index'] = [x.replace('_',' ') for x in sdm_mhs_df['system:index']]
sdm_mhs_df

In [ ]:
np.mean(sdm_mhs_df['IoU'])

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(1)
sns.scatterplot(sdm_mhs_df, x='IoU', y='system:index', ax=ax)
ax.set(ylabel=None)
plt.tight_layout()
plt.savefig(figuredir + 'sdm_mhs_iou.png', dpi=300)